In [1]:
headers = {
    'User-Agent': 'VetleBot/0.1'
}

import re
PATTERN_PAGE_LINK = re.compile(r'^(?:<ul>)?<li><a href="\/wiki\/[^"]+" title="[^"]+">([^<]+)<\/a><\/li>', re.MULTILINE)
PATTERN_NEXT_PAGE = re.compile(r'<a href="([^"]+)"[^>]*>Next page')

In [21]:
# Get the first list of pages
import requests
# Note: only *that* namespace
r = requests.get('https://pcgamingwiki.com/wiki/Special:AllPages?from=&to=&namespace=402', headers=headers)
done_processing = False
all_pages = []

import html
while done_processing is False:
    # Extract pages from list of pages
    pages = re.findall(PATTERN_PAGE_LINK, r.text)

    # Clean HTML entities
    pages = [html.unescape(page) for page in pages]
    
    # Append pages found to list
    all_pages += pages
    done_processing = True

    # Get the next list of pages page, if it exists
    m = re.search(PATTERN_NEXT_PAGE, r.text)
    if m is not None:
        # Clean it
        url = html.unescape(m.group(1))
        
        # Add prefix, if needed
        url = 'https://pcgamingwiki.com' + url if url[0] == '/' else url
        
        # Make request
        r = requests.get(url, headers=headers)
        done_processing = False

In [22]:
len(all_pages)

2293

In [5]:
# Make unique
all_pages = list(set(all_pages))
all_pages.sort() # for predictability

In [7]:
print(len(all_pages))
all_images = {}

34770


In [24]:
all_pages[1]

'Series:101 Dalmatians'

In [23]:
# for every 50, purge it
import math
batch_size = 50
batches = math.ceil(len(all_pages) / batch_size)

export_dir = '../data_pcgw'

import os
if not os.path.exists(export_dir):
    os.makedirs(export_dir)

import json
for i in range(batches):
    if i < 1:
        continue
    pages = all_pages[i * batch_size : i * batch_size + batch_size]
    pages_comb = '|'.join(pages)
    data = {
        'action': 'purge',
        'titles': pages_comb,
        'format': 'json',
    }
    r = requests.post('https://pcgamingwiki.com/w/api.php', headers=headers, data=data)
    print(f'{i + 1}/{batches}', end='\r')